In [11]:
import os
from scipy import misc
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.data_utils import get_file
from six.moves import cPickle
import sys
from keras.utils import np_utils 
import cPickle as pickle
from sklearn.cross_validation import train_test_split
import random

In [13]:
#define the working directory
cwd=os.getcwd()
(cwdtop,tail)=os.path.split(cwd)
patch_dir=os.path.join(cwdtop,'patches')
pickle_dir=os.path.join(cwdtop,'pickle')
if not os.path.exists(pickle_dir):
 os.mkdir(pickle_dir)  


In [14]:
#define a list of used labels
usedclassif = ['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']
print (usedclassif)

['consolidation', 'fibrosis', 'ground_glass', 'healthy', 'micronodules', 'reticulation']


In [15]:
#define a dictionary with labels
classif={}
i=0
for f in usedclassif:
    classif[f]=i
    i+=1

In [16]:
print classif

{'micronodules': 4, 'healthy': 3, 'ground_glass': 2, 'consolidation': 0, 'reticulation': 5, 'fibrosis': 1}


In [19]:
# list all directories under patch directory. They are representing the categories
category_list = (os.listdir(patch_dir))
##print 'taking out the item : ', category_list.pop(0)

In [20]:
# print what we have as categories
category_list

['bronchial_wall_thickening',
 'bronchiectasis',
 'consolidation',
 'cysts',
 'fibrosis',
 'ground_glass',
 'macronodules',
 'Nouveau document texte.txt',
 'peripheral_micronodules',
 'reticulation']

In [22]:
# creating variables
# list for the merged pixel data
dataset_list = []
# list of the label data
label_list = []



# go through all categories
# 
for category in category_list:
    if category in usedclassif:
        category_dir = os.path.join(patch_dir, category)
        print  'the path into the categories is: ', category_dir

        sub_categories_dir_list = (os.listdir(category_dir))
        print 'the sub categories are : ', sub_categories_dir_list

        for subCategory in sub_categories_dir_list:

            subCategory_dir = os.path.join(category_dir, subCategory)
            print  'the path into the sub categories is: '
            print subCategory_dir

            image_files = (os.listdir(subCategory_dir))

            for file in image_files:

                if file.find('.bmp') > 0:

                    # load the .bmp file into array       
                    image = misc.imread(os.path.join(subCategory_dir,file), flatten= 0)
                    # append the array to the dataset list
                    dataset_list.append(image)

                    # created rotated copies of images
                    image90 = np.rot90(image)
                    dataset_list.append(image90)

                    image180 = np.rot90(image90)
                    dataset_list.append(image180)

                    image270 = np.rot90(image180)
                    dataset_list.append(image270)

                    # append  lables 
                    label_list.append(classif[category])
                    # append also lables for rotated images
                    label_list.append(classif[category])
                    label_list.append(classif[category])
                    label_list.append(classif[category])


            

the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation
the sub categories are :  ['diffuse', 'non-relevant', 'peripheral_sub_pleural']
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\diffuse
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\non-relevant
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\consolidation\peripheral_sub_pleural
the path into the categories is:  C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\fibrosis
the sub categories are :  ['basal', 'non-relevant']
the path into the sub categories is: 
C:\Users\sylvain\Documents\boulot\startup\radiology\ILDtempo\ild-cnn\patches\fibrosis\basal
the path into the sub categories is: 
C:\Users\sylvain\Documen

In [23]:
print (len(dataset_list),len(label_list))



(1016, 1016)


In [27]:
nb_elem = 25000 
indices = []  
resultatx=[]
resultaty= [] 
while nb_elem > 0:  
    i = random.randint(0, len(dataset_list) -1)  
    while i in indices: # tant que le tirage redonne un nombre déjà choisi  
        i = random.randint(0, len(dataset_list) -1)  
    indices.append(i)  
    nb_elem = nb_elem - 1  

for index in indices:  
    resultatx.append(dataset_list[index])  
    resultaty.append(label_list[index]) 

# transform dataset list into numpy array                   
X = np.array(resultatx)
#this is already in greyscale
# use only one of the 3 color channels as greyscale info
#X = dataset[:,:, :,1]

print 'dataset shape is now: ', X.shape

# 
y = np.array(resultaty)
# sampling item 22
#y[22]


 dataset shape is now:  (1000L, 32L, 32L)


In [28]:
print (len(dataset_list),len(label_list))
print (len(resultatx),len(resultaty))

(1016, 1016)
(1000, 1000)


In [29]:
print X.shape
print y.shape

(1000L, 32L, 32L)
(1000L,)


In [30]:
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=42)
print X_train.shape
print X_val.shape
print X_test.shape
print y_train.shape
print y_test.shape
print y_val.shape
    

(500L, 32L, 32L)
(250L, 32L, 32L)
(250L, 32L, 32L)
(500L,)
(250L,)
(250L,)


In [31]:
# save the dataset and label set into serial formatted pkl 

pickle.dump(X_train, open( os.path.join(pickle_dir,"X_train.pkl"), "wb" ))
pickle.dump(X_test, open( os.path.join(pickle_dir,"X_test.pkl"), "wb" ))
pickle.dump(X_val, open(os.path.join(pickle_dir,"X_val.pkl"), "wb" ))
pickle.dump(y_train, open( os.path.join(pickle_dir,"y_train.pkl"), "wb" ))
pickle.dump(y_test, open( os.path.join(pickle_dir,"y_test.pkl"), "wb" ))
pickle.dump(y_val, open( os.path.join(pickle_dir,"y_val.pkl"), "wb" ))

In [32]:
# testing if pickls was working fine
recuperated_X_train = pickle.load( open( os.path.join(pickle_dir,"X_train.pkl"), "rb" ) )

In [33]:
recuperated_X_train

array([[[154, 155, 154, ..., 116, 115, 108],
        [153, 156, 153, ..., 125, 119, 110],
        [153, 153, 152, ..., 123, 117, 111],
        ..., 
        [156, 156, 162, ..., 155, 153, 151],
        [155, 159, 164, ..., 164, 162, 153],
        [153, 160, 161, ..., 156, 163, 161]],

       [[ 22,  22,  23, ...,  11,   9,  22],
        [ 18,  23,  23, ...,  18,  17,  17],
        [ 21,  16,  26, ...,  17,  18,  17],
        ..., 
        [ 12,  14,  30, ...,  93,  89,  79],
        [ 11,  12,  32, ...,  90,  89,  96],
        [  8,  13,  26, ...,  91, 101,  86]],

       [[ 24,  45,  56, ...,   0,   0,   1],
        [ 32,  41,  50, ...,   0,   0,   0],
        [ 27,  43,  46, ...,  17,  10,   3],
        ..., 
        [ 61,  50,  57, ...,  58,  49,  52],
        [ 48,  56,  42, ...,  53,  61,  48],
        [ 46,  48,  43, ...,  53,  58,  47]],

       ..., 
       [[154, 161, 162, ..., 137, 125, 117],
        [152, 156, 161, ..., 135, 134, 128],
        [148, 155, 158, ..., 119, 138, 